In [2]:
%matplotlib inline
import math
import random
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from collections import defaultdict
from scipy.stats.stats import pearsonr

from sklearn.cross_validation import train_test_split
from sklearn import tree
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.tree import export_graphviz
from numpy.random import randint 
from sklearn.model_selection import cross_validate
from imblearn.under_sampling import RandomUnderSampler
import collections

C:\Users\Elisa\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
#creazione di average_daily_hours, time_spenT_company, work_accident e departments
df = pd.read_csv("HR_comma_sep.csv")
df['average_daily_hours']=df['average_montly_hours']/21.5
df=df.drop(['average_montly_hours'], axis=1)

df ['time_spent_company'] = df['time_spend_company']
df=df.drop(['time_spend_company'], axis=1)

df ['work_accident'] = df['Work_accident']
df=df.drop(['Work_accident'], axis=1)

df ['departments'] = df['sales']
df=df.drop(['sales'], axis=1)

df.head()

,satisfaction_level,last_evaluation,number_project,left,promotion_last_5years,salary,average_daily_hours,time_spent_company,work_accident,departments
0,0.38,0.53,2,1,0,low,7.302326,3,0,sales
1,0.80,0.86,5,1,0,medium,12.186047,6,0,sales
2,0.11,0.88,7,1,0,medium,12.651163,4,0,sales
3,0.72,0.87,5,1,0,low,10.372093,5,0,sales
4,0.37,0.52,2,1,0,low,7.395349,3,0,sales


In [4]:
#creazione di average_daily_hours, time_spenT_company, work_accident e departments
df = pd.read_csv("HR_comma_sep.csv")
df['average_daily_hours']=df['average_montly_hours']/21.5
df=df.drop(['average_montly_hours'], axis=1)

df ['time_spent_company'] = df['time_spend_company']
df=df.drop(['time_spend_company'], axis=1)

df ['work_accident'] = df['Work_accident']
df=df.drop(['Work_accident'], axis=1)

df ['departments'] = df['sales']
df=df.drop(['sales'], axis=1)

df.head()

,satisfaction_level,last_evaluation,number_project,left,promotion_last_5years,salary,average_daily_hours,time_spent_company,work_accident,departments
0,0.38,0.53,2,1,0,low,7.302326,3,0,sales
1,0.80,0.86,5,1,0,medium,12.186047,6,0,sales
2,0.11,0.88,7,1,0,medium,12.651163,4,0,sales
3,0.72,0.87,5,1,0,low,10.372093,5,0,sales
4,0.37,0.52,2,1,0,low,7.395349,3,0,sales


In [5]:
# funzione per trasformare le variabile categoriche in numeriche dumpy/binarie (sklearn.DecisionTree funziona solo con numeri e fa solo due split per nodo)
def clean_data(df):
    # Get the unique values of departments
    departments_locs = sorted(df['departments'].unique())

    # Generate a mapping of departments from a string to a number representation        
    departments_locs_mapping = dict(zip(departments_locs, 
                                     range(0, len(departments_locs) + 1)))
    
    # Transform departments from a string to a number representation
    df['departments_val'] = df['departments'].map(departments_locs_mapping).astype(int)
    
    # Transform departments from a string to dummy variables
    df = pd.concat([df, pd.get_dummies(df['departments'], prefix='departments_val')], axis=1)
    
    # drop departments and departments_val
    df=df.drop(['departments'], axis=1)
    df=df.drop(['departments_val'], axis=1)
    
    
     # Get the unique values of salary
    salary_locs = sorted(df['salary'].unique())

    # Generate a mapping of salary from a string to a number representation        
    salary_locs_mapping = dict(zip(salary_locs, [2, 0, 1]))
    
    # Transform salary from a string to a number representation
    df['salary_val'] = df['salary'].map(salary_locs_mapping).astype(int)
    
    # Transform salary from a string to dummy variables
    df = pd.concat([df, pd.get_dummies(df['salary'], prefix='salary_val')], axis=1)
    
    # drop salary and salary_val
    df=df.drop(['salary'], axis=1)
    df=df.drop(['salary_val'], axis=1)
    return df
df= clean_data (df)

In [6]:
#creazione di un nuovo dataset da cui vengono eliminate alcune variabili che potrebbero non avere importanza nella classificazione
df_selected_feat=df.drop(['departments_val_IT', 'departments_val_RandD', 'departments_val_accounting','departments_val_hr', 'departments_val_management',      
'departments_val_marketing',        
'departments_val_product_mng',     
'departments_val_sales',          
'departments_val_support',          
'departments_val_technical'],axis=1)
df_selected_feat.head()

,satisfaction_level,last_evaluation,number_project,left,promotion_last_5years,average_daily_hours,time_spent_company,work_accident,salary_val_high,salary_val_low,salary_val_medium
0,0.38,0.53,2,1,0,7.302326,3,0,0,1,0
1,0.80,0.86,5,1,0,12.186047,6,0,0,0,1
2,0.11,0.88,7,1,0,12.651163,4,0,0,0,1
3,0.72,0.87,5,1,0,10.372093,5,0,0,1,0
4,0.37,0.52,2,1,0,7.395349,3,0,0,1,0


In [10]:
#divisione di df_selected_features in training e test
# conversione del DataFrame in un numpy array:
train_data_sel = df_selected_feat.values

# selezione delle features senza la colonna left (target)
train_sel_features = np.delete(train_data_sel,np.s_[3:4], axis=1)

# selezione del target left
train_sel_target = np.delete(train_data_sel,np.s_[0:3], axis=1)
train_sel_target = train_sel_target[:, 0]

train_sel_x, test_sel_x, train_sel_y, test_sel_y = train_test_split(train_sel_features, 
                                                    train_sel_target, 
                                                    test_size=0.33, 
                                                    random_state=0)


# DATASET BILANCIATO con random under-sampling: le classi sono bilanciate, infatti ci sono 3571 left e 3571 non left per un totale di 7142 record
train_sel_features_res, train_sel_target_res= RandomUnderSampler(ratio='majority', random_state=0).fit_sample(train_sel_features, train_sel_target)

# divisione del DATASET BILANCIATO in training e in test - tecnica holdout->1/3 per test set(train_res_x e test_res_x formano insieme tutto il dataset mentre train_res_y e test_res_y corrispondono al target)
train_sel_res_x, test_sel_res_x, train_sel_res_y, test_sel_res_y = train_test_split(train_sel_features_res, 
                                                    train_sel_target_res, 
                                                    test_size=0.33, 
                                                    random_state=0)

print(train_sel_features_res[100])
print (train_sel_target_res[100])

[ 0.95        0.81        5.          0.          5.58139535  3.          0.
  0.          1.          0.        ]
0.0


In [32]:
#prova degli alberi ottenuti in HR_CLASSIFICATION_unbal_sel con df_selected_feat BILANCIATO
clf = tree.DecisionTreeClassifier(criterion='gini', splitter='best',  max_depth=6,
                                  min_samples_split=16, max_features=10, random_state=0)
clf = clf.fit(train_sel_res_x, train_sel_res_y)
print ("DATASET BILANCIATO--->")
print (clf.feature_importances_)

print("TRANING E TEST FORMATI CON HOLDOUT->")
train_sel_res_pred = clf.predict(train_sel_res_x)
test_sel_res_pred = clf.predict(test_sel_res_x)
# Accuratezza sul training e sul test
print ("Accuratezza sul training:")
print (metrics.accuracy_score(train_sel_res_y, train_sel_res_pred))
print ("Accuratezza sul test:")
print (metrics.accuracy_score(test_sel_res_y, test_sel_res_pred))
# recall, precision, f1_measure su training e su test
print ("r, p, f1-score sul training:")
print(classification_report(train_sel_res_y, 
                            train_sel_res_pred, 
                            target_names=['Not left', 'left']))
print ("r, p, f1-score sul test:")
print(classification_report(test_sel_res_y, 
                            test_sel_res_pred, 
                            target_names=['Not left', 'left']))



DATASET BILANCIATO--->
[ 0.432126    0.09661239  0.01719227  0.          0.06462478  0.38876822
  0.          0.00067634  0.          0.        ]
TRANING E TEST FORMATI CON HOLDOUT->
Accuratezza sul training:
0.948171368861
Accuratezza sul test:
0.93975392448
r, p, f1-score sul training:
             precision    recall  f1-score   support

   Not left       0.95      0.94      0.95      2400
       left       0.94      0.95      0.95      2385

avg / total       0.95      0.95      0.95      4785

r, p, f1-score sul test:
             precision    recall  f1-score   support

   Not left       0.94      0.94      0.94      1171
       left       0.94      0.94      0.94      1186

avg / total       0.94      0.94      0.94      2357



In [62]:
#generazione diversi decision tree validati con holdout e cross-validation: per ogni depth, si prova con entropy e gini 
#e con lo stesso numero minimo di record per lo split e di max_features

crit= ['gini', 'entropy']
max_d=[2,3,4,5,6,7,8,9,10,11,12,None]

for d in max_d:
    min_s_sp=randint(2,431)
    for cr in crit:
            clf = tree.DecisionTreeClassifier(criterion=cr, splitter='best',  max_depth=d,
                                  min_samples_split=min_s_sp, max_features=10, random_state=0)
            clf = clf.fit(train_sel_res_x, train_sel_res_y)
            print ("max_depth:")
            print (d)
            print ("criterion:")
            print (cr)
            print ("min_samples_split:")
            print (min_s_sp)

            
            print ("DATASET BILANCIATO--->")
            print (clf.feature_importances_)
            
            print("TRANING E TEST FORMATI CON HOLDOUT->")
            train_sel_res_pred = clf.predict(train_sel_res_x)
            test_sel_res_pred = clf.predict(test_sel_res_x)
            # Accuratezza sul training e sul test
            print ("Accuratezza sul training:")
            print (metrics.accuracy_score(train_sel_res_y, train_sel_res_pred))
            print ("Accuratezza sul test:")
            print (metrics.accuracy_score(test_sel_res_y, test_sel_res_pred))
            # recall, precision, f1_measure su training e su test
            print ("r, p, f1-score sul training:")
            print(classification_report(train_sel_res_y, 
                                        train_sel_res_pred, 
                                        target_names=['Not left', 'left']))
            print ("r, p, f1-score sul test:")
            print(classification_report(test_sel_res_y, 
                                        test_sel_res_pred, 
                                        target_names=['Not left', 'left']))
            
            print("TRANING E TEST FORMATI CON CROSS-VALIDATION->")
            
            scores = cross_validate(clf, 
                                    train_sel_features_res, train_sel_target_res, scoring=['accuracy','recall','precision','f1'],
                                    cv=10, return_train_score=True)
            
            print ("Accuratezza test:")
            print(scores['test_accuracy'])
            print ('Accuratezza: %0.2f (+/- %0.2f)' % (scores['test_accuracy'].mean(), scores['test_accuracy'].std() * 2))
            print ("Accuratezza training:")
            print(scores['train_accuracy'])
            print ('Accuratezza: %0.2f (+/- %0.2f)' % (scores['train_accuracy'].mean(), scores['train_accuracy'].std() * 2))
            print ("Recall test:")
            print(scores['test_recall'])
            print ('Recall: %0.2f (+/- %0.2f)' % (scores['test_recall'].mean(), scores['test_recall'].std() * 2))
            print ("Recall training:")
            print(scores['train_recall'])
            print ('Recall: %0.2f (+/- %0.2f)' % (scores['train_recall'].mean(), scores['train_recall'].std() * 2))
            print ("Precision test:")
            print(scores['test_precision'])
            print ('Precision: %0.2f (+/- %0.2f)' % (scores['test_precision'].mean(), scores['test_precision'].std() * 2))
            print ("Precision training:")
            print(scores['train_precision'])
            print ('Precision: %0.2f (+/- %0.2f)' % (scores['train_precision'].mean(), scores['train_precision'].std() * 2))
            print ("f1 test:")
            print(scores['test_f1'])
            print ('F1: %0.2f (+/- %0.2f)' % (scores['test_f1'].mean(), scores['test_f1'].std() * 2))
            print ("f1 training:")
            print(scores['train_f1'])
            print ('F1: %0.2f (+/- %0.2f)' % (scores['train_f1'].mean(), scores['train_f1'].std() * 2))

max_depth:
2
criterion:
gini
min_samples_split:
93
DATASET BILANCIATO--->
[ 0.52819458  0.          0.          0.          0.          0.47180542
  0.          0.          0.          0.        ]
TRANING E TEST FORMATI CON HOLDOUT->
Accuratezza sul training:
0.878996865204
Accuratezza sul test:
0.869325413661
r, p, f1-score sul training:
             precision    recall  f1-score   support

   Not left       0.93      0.82      0.87      2400
       left       0.84      0.94      0.89      2385

avg / total       0.88      0.88      0.88      4785

r, p, f1-score sul test:
             precision    recall  f1-score   support

   Not left       0.90      0.82      0.86      1171
       left       0.84      0.91      0.88      1186

avg / total       0.87      0.87      0.87      2357

TRANING E TEST FORMATI CON CROSS-VALIDATION->
Accuratezza test:
[ 0.87430168  0.88095238  0.86414566  0.87394958  0.8697479   0.88655462
  0.8907563   0.88095238  0.86414566  0.87254902]
Accuratezza: 0.88

Accuratezza test:
[ 0.93296089  0.92857143  0.92156863  0.92717087  0.90756303  0.91876751
  0.92296919  0.92016807  0.92436975  0.92016807]
Accuratezza: 0.92 (+/- 0.01)
Accuratezza training:
[ 0.92187986  0.92237088  0.92314872  0.92252645  0.92470442  0.92299315
  0.92299315  0.92330429  0.92283759  0.92330429]
Accuratezza: 0.92 (+/- 0.00)
Recall test:
[ 0.9273743   0.91316527  0.91876751  0.92997199  0.8907563   0.9047619
  0.92436975  0.91876751  0.92717087  0.90756303]
Recall: 0.92 (+/- 0.02)
Recall training:
[ 0.91503268  0.91661481  0.91599253  0.91474798  0.91910392  0.91754823
  0.91537026  0.91599253  0.91505912  0.91723709]
Recall: 0.92 (+/- 0.00)
Precision test:
[ 0.93785311  0.94219653  0.92394366  0.92479109  0.92173913  0.93083573
  0.92178771  0.92134831  0.92200557  0.93103448]
Precision: 0.93 (+/- 0.01)
Precision training:
[ 0.92773746  0.9272899   0.92929293  0.92920354  0.92951542  0.9276502
  0.92954186  0.92958636  0.9295196   0.92850394]
Precision: 0.93 (+/- 0.00

Accuratezza test:
[ 0.94692737  0.94117647  0.93557423  0.94957983  0.93277311  0.93977591
  0.94257703  0.94117647  0.93557423  0.94117647]
Accuratezza: 0.94 (+/- 0.01)
Accuratezza training:
[ 0.94164332  0.94181705  0.94228376  0.94088363  0.94243933  0.94197262
  0.9425949   0.94181705  0.9425949   0.94181705]
Accuratezza: 0.94 (+/- 0.00)
Recall test:
[ 0.94692737  0.92717087  0.92997199  0.95518207  0.92717087  0.92997199
  0.93837535  0.93277311  0.94397759  0.92997199]
Recall: 0.94 (+/- 0.02)
Recall training:
[ 0.93993153  0.93746111  0.93683883  0.93434972  0.9390168   0.93714997
  0.93839452  0.93683883  0.93652769  0.93714997]
Recall: 0.94 (+/- 0.00)
Precision test:
[ 0.94692737  0.95389049  0.94050992  0.94459834  0.93767705  0.94857143
  0.94632768  0.94871795  0.92837466  0.9512894 ]
Precision: 0.94 (+/- 0.01)
Precision training:
[ 0.94316052  0.94569994  0.94715319  0.94672131  0.94548872  0.9462771
  0.94634452  0.94626021  0.9480315   0.9459799 ]
Precision: 0.95 (+/- 0.0

Accuratezza test:
[ 0.95670391  0.94817927  0.94957983  0.96358543  0.95098039  0.95658263
  0.95518207  0.95378151  0.95378151  0.95658263]
Accuratezza: 0.95 (+/- 0.01)
Accuratezza training:
[ 0.95580454  0.95644057  0.95644057  0.95472931  0.95581829  0.95550716
  0.95566273  0.95581829  0.956285    0.95535159]
Accuratezza: 0.96 (+/- 0.00)
Recall test:
[ 0.91899441  0.90196078  0.9047619   0.92997199  0.90756303  0.91596639
  0.91316527  0.91036415  0.91596639  0.92156863]
Recall: 0.91 (+/- 0.02)
Recall training:
[ 0.91565515  0.91568139  0.91599253  0.91257001  0.91661481  0.9141257
  0.91443684  0.91474798  0.91599253  0.91350342]
Recall: 0.91 (+/- 0.00)
Precision test:
[ 0.9939577   0.99382716  0.99384615  0.996997    0.99386503  0.99695122
  0.9969419   0.99693252  0.99090909  0.99096386]
Precision: 0.99 (+/- 0.00)
Precision training:
[ 0.99560068  0.99695122  0.99661476  0.99660211  0.99459824  0.99660787
  0.99660902  0.99661017  0.9962775   0.99694397]
Precision: 1.00 (+/- 0.0

Accuratezza test:
[ 0.94692737  0.93837535  0.93977591  0.95238095  0.92997199  0.94257703
  0.94397759  0.94257703  0.94397759  0.94257703]
Accuratezza: 0.94 (+/- 0.01)
Accuratezza training:
[ 0.9431995   0.94383945  0.94383945  0.94243933  0.94446173  0.94337274
  0.94321717  0.94337274  0.94368388  0.94321717]
Accuratezza: 0.94 (+/- 0.00)
Recall test:
[ 0.89944134  0.87955182  0.88235294  0.9047619   0.86554622  0.88795518
  0.8907563   0.88795518  0.89635854  0.8907563 ]
Recall: 0.89 (+/- 0.02)
Recall training:
[ 0.88951136  0.88985688  0.89016801  0.88736777  0.89296826  0.88892346
  0.88861232  0.88892346  0.89016801  0.88861232]
Recall: 0.89 (+/- 0.00)
Precision test:
[ 0.99382716  0.9968254   0.99683544  1.          0.99356913  0.99685535
  0.9968652   0.99685535  0.99071207  0.99375   ]
Precision: 1.00 (+/- 0.00)
Precision training:
[ 0.99651325  0.99755842  0.99721157  0.9972028   0.99549081  0.99755587
  0.99755501  0.99755587  0.99686411  0.99755501]
Precision: 1.00 (+/- 0.

Accuratezza test:
[ 0.96787709  0.95378151  0.95518207  0.96638655  0.95098039  0.96078431
  0.96778711  0.95798319  0.95658263  0.95378151]
Accuratezza: 0.96 (+/- 0.01)
Accuratezza training:
[ 0.96389667  0.96453018  0.96453018  0.96250778  0.96437461  0.96313006
  0.9639079   0.96406347  0.96328563  0.96406347]
Accuratezza: 0.96 (+/- 0.00)
Recall test:
[ 0.95530726  0.91876751  0.91876751  0.93557423  0.91596639  0.93277311
  0.94957983  0.92997199  0.93557423  0.93277311]
Recall: 0.93 (+/- 0.02)
Recall training:
[ 0.93899782  0.93652769  0.93777225  0.93403858  0.93932794  0.93466086
  0.93652769  0.93559428  0.93217175  0.94026136]
Recall: 0.94 (+/- 0.00)
Precision test:
[ 0.97994269  0.98795181  0.99093656  0.99701493  0.98493976  0.98813056
  0.98546512  0.9851632   0.97660819  0.97368421]
Precision: 0.98 (+/- 0.01)
Precision training:
[ 0.98820832  0.99208965  0.99079553  0.9904322   0.98886341  0.99109205
  0.99078341  0.99208182  0.99402787  0.98725907]
Precision: 0.99 (+/- 0.

In [36]:
#Accuratezza di training e test DEL DATASET BILANCIATO all'aumentare di max_deph da 2 a 12 per parametri trovati nella prima prova 
#(PER VEDERE SE E QUANDO SI ENTRA IN OVERFITTING)
from sklearn.metrics import confusion_matrix
max_d=[2,3,4,5,6,7,8,9,10,11,12]

errors_training=[]
errors_test=[]
for d in max_d:
    clf = tree.DecisionTreeClassifier(criterion='gini', splitter='best',  max_depth=d,
                                      min_samples_split=10, max_features=10, random_state=0)
    clf= clf.fit(train_sel_res_x, train_sel_res_y)
    if(d==6):
        clf_def=clf

    print ("Max_depth:")
    print(d)
    print ("DATASET SBILANCIATO--->")
    print (clf.feature_importances_)

    print("TRANING E TEST FORMATI CON HOLDOUT->")
    train_sel_res_pred = clf.predict(train_sel_res_x)
    cm=confusion_matrix(train_sel_res_y, train_sel_res_pred)
    errors_training.append(cm[0][1]+cm[1][0])
    test_sel_res_pred = clf.predict(test_sel_res_x)
    cm_t=confusion_matrix(test_sel_res_y, test_sel_res_pred)
    errors_test.append(cm_t[0][1]+cm_t[1][0])
    # Accuratezza sul training e sul test
    print ("Accuratezza sul training:")
    print (metrics.accuracy_score(train_sel_res_y, train_sel_res_pred))
    print ("Accuratezza sul test:")
    print (metrics.accuracy_score(test_sel_res_y, test_sel_res_pred))
    # recall, precision, f1_measure su training e su test
    print ("r, p, f1-score sul training:")
    print(classification_report(train_sel_res_y, 
                            train_sel_res_pred, 
                            target_names=['Not left', 'left']))
    print ("r, p, f1-score sul test:")
    print(classification_report(test_sel_res_y, 
                            test_sel_res_pred, 
                            target_names=['Not left', 'left']))

    print("TRANING E TEST FORMATI CON CROSS-VALIDATION->")

    scores = cross_validate(clf, 
                        train_sel_features_res, train_sel_target_res, scoring=['accuracy','recall','precision','f1'],
                        cv=10, return_train_score=True)

    print ("Accuratezza test:")
    print(scores['test_accuracy'])
    print ('Accuratezza: %0.2f (+/- %0.2f)' % (scores['test_accuracy'].mean(), scores['test_accuracy'].std() * 2))
    print ("Accuratezza training:")
    print(scores['train_accuracy'])
    print ('Accuratezza: %0.2f (+/- %0.2f)' % (scores['train_accuracy'].mean(), scores['train_accuracy'].std() * 2))
    print ("Recall test:")
    print(scores['test_recall'])
    print ('Recall: %0.2f (+/- %0.2f)' % (scores['test_recall'].mean(), scores['test_recall'].std() * 2))
    print ("Recall training:")
    print(scores['train_recall'])
    print ('Recall: %0.2f (+/- %0.2f)' % (scores['train_recall'].mean(), scores['train_recall'].std() * 2))
    print ("Precision test:")
    print(scores['test_precision'])
    print ('Precision: %0.2f (+/- %0.2f)' % (scores['test_precision'].mean(), scores['test_precision'].std() * 2))
    print ("Precision training:")
    print(scores['train_precision'])
    print ('Precision: %0.2f (+/- %0.2f)' % (scores['train_precision'].mean(), scores['train_precision'].std() * 2))
    print ("f1 test:")
    print(scores['test_f1'])
    print ('F1: %0.2f (+/- %0.2f)' % (scores['test_f1'].mean(), scores['test_f1'].std() * 2))
    print ("f1 training:")
    print(scores['train_f1'])
    print ('F1: %0.2f (+/- %0.2f)' % (scores['train_f1'].mean(), scores['train_f1'].std() * 2))
    
print(errors_training)
print(errors_test)


Max_depth:
2
DATASET SBILANCIATO--->
[ 0.52819458  0.          0.          0.          0.          0.47180542
  0.          0.          0.          0.        ]
TRANING E TEST FORMATI CON HOLDOUT->
Accuratezza sul training:
0.878996865204
Accuratezza sul test:
0.869325413661
r, p, f1-score sul training:
             precision    recall  f1-score   support

   Not left       0.93      0.82      0.87      2400
       left       0.84      0.94      0.89      2385

avg / total       0.88      0.88      0.88      4785

r, p, f1-score sul test:
             precision    recall  f1-score   support

   Not left       0.90      0.82      0.86      1171
       left       0.84      0.91      0.88      1186

avg / total       0.87      0.87      0.87      2357

TRANING E TEST FORMATI CON CROSS-VALIDATION->
Accuratezza test:
[ 0.87430168  0.88095238  0.86414566  0.87394958  0.8697479   0.88655462
  0.8907563   0.88095238  0.86414566  0.87254902]
Accuratezza: 0.88 (+/- 0.02)
Accuratezza training:
[ 0

Accuratezza test:
[ 0.95111732  0.94677871  0.94257703  0.94957983  0.92997199  0.94257703
  0.94257703  0.94117647  0.93277311  0.94397759]
Accuratezza: 0.94 (+/- 0.01)
Accuratezza training:
[ 0.94568939  0.94601742  0.94695084  0.94601742  0.94803983  0.94695084
  0.94648413  0.94695084  0.94710641  0.94648413]
Accuratezza: 0.95 (+/- 0.00)
Recall test:
[ 0.96089385  0.95238095  0.94957983  0.96078431  0.92997199  0.94397759
  0.95518207  0.95238095  0.94957983  0.94957983]
Recall: 0.95 (+/- 0.02)
Recall training:
[ 0.95487084  0.95332918  0.95519602  0.95301805  0.9598631   0.95830741
  0.95768513  0.95519602  0.95052894  0.9542626 ]
Recall: 0.96 (+/- 0.01)
Precision test:
[ 0.94246575  0.94182825  0.93646409  0.93972603  0.92997199  0.94134078
  0.93169399  0.93150685  0.91869919  0.93905817]
Precision: 0.94 (+/- 0.01)
Precision training:
[ 0.93765281  0.93958908  0.93970003  0.93985885  0.93768997  0.93702464
  0.93670116  0.93970003  0.94406675  0.93964461]
Precision: 0.94 (+/- 0.

Accuratezza test:
[ 0.97067039  0.96358543  0.95798319  0.95938375  0.95378151  0.96358543
  0.9789916   0.96078431  0.95938375  0.9719888 ]
Accuratezza: 0.96 (+/- 0.01)
Accuratezza training:
[ 0.97478992  0.97682016  0.97790915  0.97837586  0.97790915  0.97744244
  0.97682016  0.97604231  0.97573118  0.97682016]
Accuratezza: 0.98 (+/- 0.00)
Recall test:
[ 0.97206704  0.95238095  0.93277311  0.93837535  0.92717087  0.95238095
  0.97759104  0.95238095  0.96078431  0.96358543]
Recall: 0.95 (+/- 0.03)
Recall training:
[ 0.95860566  0.96266335  0.96608587  0.9639079   0.96453018  0.96421904
  0.96266335  0.96328563  0.96172993  0.96297449]
Recall: 0.96 (+/- 0.00)
Precision test:
[ 0.96935933  0.97421203  0.98230088  0.97953216  0.97928994  0.97421203
  0.98033708  0.96866097  0.95810056  0.98005698]
Precision: 0.97 (+/- 0.01)
Precision training:
[ 0.99067224  0.99071406  0.98948375  0.99263057  0.99104859  0.99041227
  0.99071406  0.98850575  0.98943662  0.9904    ]
Precision: 0.99 (+/- 0.